In [1]:
from glob import glob
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# 이미지를 인코딩하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [12]:
# 이미지 경로를 받아 퀴즈를 만드는 함수
def image_quiz(image_path):
    base64_image = encode_image(image_path)

    quiz_prompt = """
    제공한 이미지를 바탕으로, 다음과 같은 양식으로 퀴즈를 만들어 주세요.
    정답은 (1) ~ (4) 중 하나만 해당하도록 출제하세요.
    아래는 예시입니다.
    ----- 예시 -----

    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
    - (1) 베이커리에서 사람들이 빵을 사는 모습이 담겨 있습니다.
    - (2) 맨 앞에 서 있는 사람은 빨간색 셔츠를 입었습니다.
    - (3) 기차를 타기 위해 줄을 서 있는 사람들이 있습니다.
    - (4) 점원은 노란색 티셔츠를 입었습니다.

    정담: (4) 점원은 노란색 티셔츠가 아닌 파란색 티셔츠를 입었습니다.
    (주의: 정답은 (1)~(4) 중 하나만 선택하도록 출제하세요)
    =====
    """

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": quiz_prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        },
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
    )

    return response.choices[0].message.content    

In [5]:
q = image_quiz("../data/images/busan_dive.jpg")
print(q)

Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
- (1) 많은 사람들이 테이블에 앉아 무언가를 작업하고 있는 모습입니다.
- (2) 이미지의 배경에는 "DIVE 2024 IN BUSAN"이라는 문구가 보입니다.
- (3) 사람들 사이에 커다란 빨간색 공이 놓여 있습니다.
- (4) 천장 구조가 노출되어 있는 실내 공간입니다.

정답: (3) 사람들 사이에 커다란 빨간색 공은 보이지 않습니다.
None


In [13]:
# 여러 이미지와 관련된 문제를 생성하고 그 결과를 문제집으로 만드는 코드를 작성합니다.
txt = '' # ①  문제들을 계속 붙여 나가기 위해 빈 문자열 선언
no = 1 # 문제 번호를 위해 선언
for g in glob('../data/images/*.jpg'):  # ②
    try:
        q = image_quiz(g) # 문제 출제 (가끔 GPT에서 에러 발생할 수 있으므로 try문 사용)
    except Exception as e:
        print(e)
        continue

    divider = f'## 문제 {no}\n\n'
    print(divider)
    
    txt += divider  # ③
    # 파일명 추출해 이미지 링크 만들기
    filename = os.path.basename(g) # ③ 마크다운에 표시할 이미지 파일 경로 설정   
    txt += f'![image]({filename})\n\n' # ③

    # 문제 추가
    print(q)
    txt += q + '\n\n---------------------\n\n'
    # ④ 마크다운 파일로 저장
    with open('../data/images/image_quiz.md', 'w', encoding='utf-8') as f:
        f.write(txt)
    
    no += 1 # 문제 번호 증가

## 문제 1


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?

- (1) 많은 사람들이 노트북 앞에 앉아 있습니다.
- (2) 벽에 '비상구 EXIT'라고 적힌 표지판이 보입니다.
- (3) 천장에는 여러 개의 조명이 설치되어 있습니다.
- (4) 참가자들 모두 동일한 티셔츠를 입고 있습니다.

정답: (4) 참가자들이 모두 동일한 티셔츠를 입고 있는 것은 아닙니다.
## 문제 2


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
- (1) 노란색 조형물이 중앙에 있습니다.
- (2) 건물 외벽에 "Local Stitch"라는 글자가 보입니다.
- (3) 배경의 건물은 모두 빨간색 벽돌로 되어 있습니다.
- (4) 이미지의 오른쪽 아래에는 검은색 안내판이 있습니다.

정답: (3) 배경의 건물은 모두 빨간색 벽돌로 되어 있지 않습니다.
## 문제 3


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
- (1) 노란색 벽이 있는 카페 내부입니다.
- (2) 사람들은 주문을 하고 있는 모습입니다.
- (3) 테이블과 의자들이 비어 있습니다.
- (4) 창문 밖으로 해변이 보입니다.

정답: (4) 창문 밖으로 해변이 보이지 않습니다.
## 문제 4


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
- (1) 건물 외관은 벽돌로 이루어져 있습니다.
- (2) 건물 1층에는 카페가 위치해 있습니다.
- (3) 건물 옆에 파란색 차가 주차되어 있습니다.
- (4) 건물 옥상에는 검은색 지붕이 있습니다.

정답: (3) 건물 옆에 파란색 차가 주차되어 있지 않습니다.
## 문제 5


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
- (1) 다양한 종류의 빵이 가득 진열되어 있습니다.
- (2) 점원들은 흰색 모자를 쓰고 있습니다.
- (3) 빵 가격표가 유리 진열대 위에 놓여 있습니다.
- (4) 점원 중 한 명은 노란색 옷을 입고 있습니다.

정답: (4) 점원은 노란색 옷을 입고 있